In [80]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import statsmodels.api as sm

In [ ]:

ds = pd.read_json('quotes-2019-nytimes.json', lines=True)

In [85]:
# just for the purpose to be abel to code smtg

quote_num = ds.shape[0]
dis_num = 4133

ds['disaster']=np.random.randint(0, dis_num, quote_num)
ds.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,disaster
0,2019-04-17-024782,"It is not a low-income immigration,",James Fisher,"[Q16213953, Q20707104, Q43143598, Q58886302, Q...",2019-04-17 13:31:18,1,"[[James Fisher, 0.7475], [None, 0.2525]]",[https://www.nytimes.com/2019/04/17/realestate...,E,1721
1,2019-04-02-001128,a champion figure skater switching to roller s...,John Updike,[Q105756],2019-04-02 14:58:33,2,"[[John Updike, 0.5856], [None, 0.4144]]",[https://www.nytimes.com/2019/04/02/opinion/vl...,E,1982
2,2019-05-09-055187,It makes it much more difficult for him to mak...,None,[],2019-05-09 18:11:29,1,"[[None, 0.6493], [President Bill Clinton, 0.27...",[http://mobile.nytimes.com/2019/05/09/world/as...,E,1582
3,2019-10-31-056366,"It puts me in a predicament,",Xavier Becerra,[Q1855840],2019-10-31 16:45:15,3,"[[Xavier Becerra, 0.9065], [None, 0.0909], [St...",[http://www.nytimes.com/2019/10/31/technology/...,E,3135
4,2019-01-04-001792,A Pile of Leaves.,None,[],2019-01-04 10:00:07,1,"[[None, 0.8737], [Jason Fulford, 0.1263]]",[https://www.nytimes.com/2019/01/04/books/revi...,E,2058


# Influencing factors in media coverage

Our hypothesis is that we can observe come connection between the volume of the appearance of a disaster in the media and the characteristics of the disasters. To be concrete we would expect that there are more quotes about disasters with more casualities. In this section our aim is to determine these factors in the observed data of diasaters and examine their contribution to media coverage.

## Reaching the disasters

As we have seen in the previous milestone, we can assume that to a specific disaster the quote realted to it will contain the name of the place and the type of the disaster. So our first goal is to assign the specific quotes according to this assumption to the adequate events.

Step 1: Collect the names and places of the biggest disasters.

In [100]:
disasters = pd.read_csv("datasets/emdat/emdat_public_2021_11_06_clean.csv")
disasters.head()

,Unnamed: 0,Dis No,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Continent,OFDA Response,Appeal,...,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),human_readable_name,start_date,end_date
0,0,2010-0017-HTI,Earthquake,Ground movement,NaN,Haiti,HTI,Americas,True,NaN,...,300000.0,3400000.0,NaN,3700000.0,11500000.0,200000.0,8000000.0,2010 Earthquake in Haiti,2010-01-01,2010-12-31
1,5,2010-0372-RUS,Extreme temperature,Heat wave,NaN,Russian Federation (the),RUS,Europe,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,400000.0,2010 Extreme temperature in Russian Federatio...,2010-01-01,2010-12-31
2,10,2010-9082-SOM,Drought,Drought,NaN,Somalia,SOM,Africa,False,Yes,...,NaN,4000000.0,NaN,4000000.0,NaN,NaN,NaN,2010 Drought in Somalia,2010-01-01,2011-12-31
3,11,2011-0082-JPN,Earthquake,Tsunami,NaN,Japan,JPN,Asia,True,NaN,...,5933.0,362887.0,NaN,368820.0,NaN,37500000.0,210000000.0,2011 Earthquake in Japan,2011-01-01,2011-12-31
4,16,2015-0144-NPL,Earthquake,Ground movement,NaN,Nepal,NPL,Asia,True,NaN,...,17932.0,5621790.0,NaN,5639722.0,NaN,100000.0,5174000.0,2015 Earthquake in Nepal,2015-01-01,2015-12-31


Step 2: Collect the specific quotes



In [14]:
def contain_all_words_regex(words):
    return "".join([f"(?=.*{word})" for word in words])

def quotes_that_contain_all_words(words):
    return dataset[dataset.quotation.str.lower().str.contains(contain_all_words_regex(words), regex=True)]

## Determine the contibuting factors

We should find first the relevant columns in the disaster dataset which can be tested that it has an effect of the volume numbers. 

## WARNING 

To run the below cell, it takes hours, so dont run it if you dont want to recalculate it, you can find the data in a pickle.

In [106]:
# we have a dataset about the disasters with word developpment indicators
dis_wdi = pd.read_csv('wdi_with_disasters.csv')

indicators = dis_wdi['indicator_name'].unique()

emp = np.empty(dis_num)
emp[:] = np.NaN

for indi in indicators: 
    disasters[f'{indi}'] = emp


for i in tqdm(range(dis_num)): # iterate over disasters
    
    for indi in indicators: # iterate over indicators
     
        curr_dis = disasters.iloc[i]
        name = curr_dis['human_readable_name']
        year = int(curr_dis['start_date'][:4])

        val = dis_wdi[(dis_wdi['human_readable_name']==name) & (dis_wdi['indicator_name']==indi)][str(year)].values

        if len(val) == 0:
            pass
        else: 
            disasters.loc[disasters['human_readable_name']==name, f'{indi}'] = val[0]

            

disasters.head()

C:\Users\jhild\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 4133/4133 [1:08:43<00:00,  1.00it/s]


,Unnamed: 0,Dis No,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Continent,OFDA Response,Appeal,...,"School enrollment, primary (% gross)","School enrollment, primary and secondary (gross), gender parity index (GPI)","School enrollment, secondary (% gross)",Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %)
0,0,2010-0017-HTI,Earthquake,Ground movement,NaN,Haiti,HTI,Americas,True,NaN,...,NaN,NaN,NaN,42.222222,27750.0,NaN,NaN,134.0,12.512662,3.618578
1,5,2010-0372-RUS,Extreme temperature,Heat wave,NaN,Russian Federation (the),RUS,Europe,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,2010-9082-SOM,Drought,Drought,NaN,Somalia,SOM,Africa,False,Yes,...,NaN,NaN,NaN,26.666667,637660.0,NaN,NaN,NaN,NaN,7.073623
3,11,2011-0082-JPN,Earthquake,Tsunami,NaN,Japan,JPN,Asia,True,NaN,...,NaN,NaN,NaN,NaN,377955.0,NaN,NaN,NaN,NaN,0.097377
4,16,2015-0144-NPL,Earthquake,Ground movement,NaN,Nepal,NPL,Asia,True,NaN,...,144.177673,1.05463,67.843689,72.222233,147180.0,14.686305,NaN,17.0,8.441650,2.445982


In [108]:
import pickle as pickle
with open(  f'disasters'+'.pickle', 'wb' ) as handle:
    pickle.dump(disasters, handle, protocol=pickle.HIGHEST_PROTOCOL)

#  NOW YOU CAN RUN THE CODES

In [109]:

disasters = pd.read_pickle('disasters.pickle')
disasters.head()

,Unnamed: 0,Dis No,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Continent,OFDA Response,Appeal,...,"School enrollment, primary (% gross)","School enrollment, primary and secondary (gross), gender parity index (GPI)","School enrollment, secondary (% gross)",Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %)
0,0,2010-0017-HTI,Earthquake,Ground movement,NaN,Haiti,HTI,Americas,True,NaN,...,NaN,NaN,NaN,42.222222,27750.0,NaN,NaN,134.0,12.512662,3.618578
1,5,2010-0372-RUS,Extreme temperature,Heat wave,NaN,Russian Federation (the),RUS,Europe,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,2010-9082-SOM,Drought,Drought,NaN,Somalia,SOM,Africa,False,Yes,...,NaN,NaN,NaN,26.666667,637660.0,NaN,NaN,NaN,NaN,7.073623
3,11,2011-0082-JPN,Earthquake,Tsunami,NaN,Japan,JPN,Asia,True,NaN,...,NaN,NaN,NaN,NaN,377955.0,NaN,NaN,NaN,NaN,0.097377
4,16,2015-0144-NPL,Earthquake,Ground movement,NaN,Nepal,NPL,Asia,True,NaN,...,144.177673,1.05463,67.843689,72.222233,147180.0,14.686305,NaN,17.0,8.441650,2.445982


In [112]:
disasters.columns
unneeded_columns = ['Unnamed: 0', 'Dis No', 'Disaster Type', 'Disaster Subtype',
       'Event Name', 'Country', 'ISO', 'Continent', 'Appeal', 'start_date', 'end_date', 'human_readable_name']
work_ds = disasters.drop(unneeded_columns, axis=1)
work_ds

,OFDA Response,Declaration,Aid Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,Local Time,Total Deaths,No Injured,...,"School enrollment, primary (% gross)","School enrollment, primary and secondary (gross), gender parity index (GPI)","School enrollment, secondary (% gross)",Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %)
0,True,NaN,NaN,7.0,Richter,18.443,-72.571,16:53,222570.0,300000.0,...,NaN,NaN,NaN,42.222222,27750.0,NaN,NaN,134.0,12.512662,3.618578
1,False,NaN,NaN,40.0,°C,NaN,NaN,NaN,55736.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,Yes,NaN,NaN,Km2,NaN,NaN,NaN,20000.0,NaN,...,NaN,NaN,NaN,26.666667,637660.0,NaN,NaN,NaN,NaN,7.073623
3,True,NaN,NaN,9.0,Richter,38.297,142.373,14:46,19846.0,5933.0,...,NaN,NaN,NaN,NaN,377955.0,NaN,NaN,NaN,NaN,0.097377
4,True,NaN,NaN,8.0,Richter,28.230,84.731,11:56,8831.0,17932.0,...,144.177673,1.05463,67.843689,72.222233,147180.0,14.686305,NaN,17.0,8.441650,2.445982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,False,NaN,NaN,NaN,Km2,NaN,NaN,NaN,NaN,NaN,...,89.191139,0.72831,20.555040,50.000033,NaN,NaN,NaN,58.0,NaN,3.935693
4129,False,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,107.0,...,115.448257,1.04883,94.093086,56.666667,NaN,NaN,NaN,16.0,3.309866,1.145440
4130,False,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,394.0,...,115.448257,1.04883,94.093086,56.666667,NaN,NaN,NaN,16.0,3.309866,1.145440
4131,False,NaN,NaN,73269.0,Km2,-33.749,-55.922,NaN,NaN,NaN,...,104.305038,1.05744,123.003822,77.777800,NaN,18.154268,NaN,6.5,NaN,0.456481


## Determine the volume of media coverage

The best ways to measure the media coverage of a disaster is to count the number of occurrences in the quotes and the length between the first and last appearance. 

In this part we will calculate these indicators.

In [126]:
media_length = [] # last 
media_volume = [] # number of quotes
media_1st_year = []
for i in range(dis_num): 
    d = ds[ds['disaster']==i]

    # calculate number of quotes
    v = d['numOccurrences'].values.sum()
    media_volume.append(v)

    # calculate length of appearance (in days)
    l = (d['date'].max() - d['date'].min()).days
    media_length.append(l)

    # number of quotes in the 1st year
    d['delta_date'] = d['date']-d['date'].min()
    d['delta_days'] = d['delta_date'].apply(lambda x: x.days)
    y = d[d['delta_days']<=366]['numOccurrences'].values.sum()
    media_1st_year.append(y)

work_ds['Number of Quotes'] = media_volume
work_ds['Length of Appearance'] = media_length
work_ds['Number of Quotes in 1st year'] = media_1st_year

C:\Users\jhild\AppData\Local\Temp/ipykernel_22572/1573588403.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['delta_date'] = d['date']-d['date'].min()
C:\Users\jhild\AppData\Local\Temp/ipykernel_22572/1573588403.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['delta_days'] = d['delta_date'].apply(lambda x: x.days)


In [127]:
work_ds.corr(method='pearson')

,OFDA Response,Aid Contribution,Dis Mag Value,Latitude,Longitude,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,...,Statistical Capacity Score (Overall Average) (scale 0 - 100),Surface area (sq. km),Tax revenue (% of GDP),Terrestrial and marine protected areas (% of total territorial area),Time required to start a business (days),"Total debt service (% of exports of goods, services and primary income)",Urban population growth (annual %),Number of Quotes,Length of Appearance,Number of Quotes in 1st year
OFDA Response,1.000000,-0.183172,-0.015134,-0.070329,-0.044346,0.086092,0.056934,0.036840,0.144904,0.040665,...,-0.053648,-0.115532,0.054338,-0.097628,0.043366,-0.019156,0.055506,-0.008948,-0.008374,-0.008948
Aid Contribution,-0.183172,1.000000,-0.099436,-0.089780,0.169182,0.007663,0.965971,-0.030494,1.000000,-0.027709,...,0.272855,0.260729,-0.104229,-1.000000,-0.008213,0.008634,-0.075359,0.022340,-0.299459,0.022340
Dis Mag Value,-0.015134,-0.099436,1.000000,-0.029771,-0.058100,-0.005762,0.006463,0.114280,-0.022865,0.069444,...,-0.056681,0.085823,-0.051802,0.034794,0.033590,0.017140,0.091583,0.023253,-0.018038,0.023253
Latitude,-0.070329,-0.089780,-0.029771,1.000000,0.037977,-0.001480,0.003647,0.008588,-0.016990,0.006912,...,0.003369,0.045200,-0.068915,-0.019891,-0.088605,-0.152483,-0.015993,0.034672,0.006366,0.034672
Longitude,-0.044346,0.169182,-0.058100,0.037977,1.000000,-0.027193,-0.047285,0.043979,-0.029221,0.039410,...,-0.088147,0.130940,-0.083286,-0.120521,-0.088619,-0.095040,0.161016,0.004157,0.010389,0.004157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Total debt service (% of exports of goods, services and primary income)",-0.019156,0.008634,0.017140,-0.152483,-0.095040,-0.004134,0.009850,-0.041704,-0.032950,-0.037042,...,0.422758,-0.161701,0.172529,0.018637,-0.097438,1.000000,-0.415738,-0.021369,0.017125,-0.021369
Urban population growth (annual %),0.055506,-0.075359,0.091583,-0.015993,0.161016,0.025953,0.035491,0.027534,-0.061327,0.023240,...,-0.466408,0.080368,-0.367835,-0.143708,0.168993,-0.415738,1.000000,-0.024865,-0.024921,-0.024865
Number of Quotes,-0.008948,0.022340,0.023253,0.034672,0.004157,-0.012522,-0.000979,0.020545,-0.035482,0.017411,...,-0.010475,-0.002617,0.039411,0.026722,-0.003936,-0.021369,-0.024865,1.000000,0.033413,1.000000
Length of Appearance,-0.008374,-0.299459,-0.018038,0.006366,0.010389,-0.002136,0.012599,0.017943,-0.006088,0.015611,...,0.022768,0.021023,0.015329,0.042073,-0.001356,0.017125,-0.024921,0.033413,1.000000,0.033413


In [130]:
model = sm.OLS(work_ds['Urban population growth (annual %)'], work_ds['Number of Quotes in 1st year'])
results = model.fit()
print(results.summary())

                                         OLS Regression Results                                        
Dep. Variable:     Urban population growth (annual %)   R-squared (uncentered):                     nan
Model:                                            OLS   Adj. R-squared (uncentered):                nan
Method:                                 Least Squares   F-statistic:                                nan
Date:                                Sun, 12 Dec 2021   Prob (F-statistic):                         nan
Time:                                        14:55:51   Log-Likelihood:                             nan
No. Observations:                                4133   AIC:                                        nan
Df Residuals:                                    4132   BIC:                                        nan
Df Model:                                           1                                                  
Covariance Type:                            nonrobust           

## Visualisations

This is the right place to make some nice visualizations for the datastory. 